<a href="https://colab.research.google.com/github/farshidbalan/FluentPython/blob/master/Chapter14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Iterables, Iterators, and Generators

## Sentence Take #1: A Sequence of Words

We’ll start our exploration of iterables by implementing a Sentence class: you give its
constructor a string with some text, and then you can iterate word by word. The first
version will implement the sequence protocol, and it’s iterable because all sequences are
iterable, as we’ve seen before, but now we’ll see exactly why.

In [0]:
# Example 14-1 shows a Sentence class that extracts words from a text by index.
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
  def __init__(self, text):
    self.text = text
    self.words = RE_WORD.findall(text) # re.findall returns a list with all nonoverlapping matches of the regular
                                      # expression, as a list of strings.
    
  def __getitem__(self, index):
    return self.words[index]  # self.words holds the result of .findall, so we simply return the word at the given index
  
  def __len__(self):
    return len(self.words) # To complete the sequence protocol, we implement __len__—but it is not needed to make an iterable object.
  
  def __repr__(self):
    return f'Sentence({reprlib.repr(self.text)})' # reprlib.repr is a utility function to generate abbreviated string representations
                                                  # of data structures that can be very large.


In [0]:
s = Sentence('The time has come, the walrus said')
s

Sentence('The time has...e walrus said')

## Why Sequences Are Iterable: The iter Function

Whenever the interpreter needs to iterate over an object x, it automatically calls iter(x).
The iter built-in function:
1. Checks whether the object implements \_\_iter\_\_, and calls that to obtain an iterator.
2. If \_\_iter\_\_ is not implemented, but \_\_getitem\_\_ is implemented, Python creates
an iterator that attempts to fetch items in order, starting from index 0 (zero).
3. If that fails, Python raises TypeError, usually saying “C object is not iterable,” where
C is the class of the target object.

That is why any Python sequence is iterable: they all implement \_\_getitem\_\_. In fact,
the standard sequences also implement \_\_iter\_\_, and yours should too, because the
special handling of \_\_getitem\_\_ exists for backward compatibility reasons and may be
gone in the future (although it is not deprecated as I write this).

### Remark

As of Python 3.4, the most accurate way to check whether an ob‐
ject x is iterable is to call iter(x) and handle a TypeError excep‐
tion if it isn’t. This is more accurate than using isinstance(x,
abc.Iterable), because iter(x) also considers the legacy
\_\_getitem\_\_ method, while the Iterable ABC does not.

## Iterator and Iterable

iterable
Any object from which the iter built-in function can obtain an iterator. Objects
implementing an \_\_iter\_\_ method returning an iterator are iterable. Sequences are always iterable; as are objects implementing a \_\_getitem\_\_ method that takes 0-based indexes.

In [0]:
# If there was no for statement and we had to emulate the for machinery by hand with a while loop, this is what we’d have to write:
s = 'ABC'
it = iter(s)

while True:
  try:
    print(next(it))
  except StopIteration:
    del it
    break

A
B
C


The standard interface for an iterator has two methods:
1. \_\_next\_\_
Returns the next available item, raising StopIteration when there are no more
items.

2. \_\_iter\_\_ Returns self; this allows iterators to be used where an iterable is expected, for
example, in a for loop.

In [0]:
# Example 14-3. abc.Iterator class; extracted from Lib/_collections_abc.py
class Iterator(Iterable):
  
  __slots__ = ()
  @abstractmethod
  def __next__(self):
    # 'Return the next item from the iterator. When exhausted, raise StopIteration'
    raise StopIteration
  
  def __iter__(self):
    return self
  
  @classmethod
  def __subclasshook__(cls, C): understand
    if cls is Iterator:
      if (any("__next__" in B.__dict__ for B in C.__mro__) and
          any("__iter__" in B.__dict__ for B in C.__mro__)):
        return True
    return NotImplemented

### remark

Iterators in Python aren't a matter of type but of protocol. A large and changing number of builtin types implement *some* flavor of
iterator. Don't check the type! Use hasattr to check for both "\_\_iter\_\_" and "\_\_next\_\_" attributes instead.

## Best way to check if an object is iterable

The best way to check if an
object x is an iterator is to call isinstance(x, abc.Iterator).
Thanks to Iterator.\_\_subclasshook\_\_, this test works even if the
class of x is not a real or virtual subclass of Iterator.

## Sentence Take #2: A Classic Iterator

In [0]:
# Example 14-4. sentence_iter.py: Sentence implemented using the Iterator pattern
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
  
  def __init__(self, text):
    self.text = text
    self.words = RE_WORD.findall(text)
    
  def __repr(self):
    return f'Sentence({reprlib.repr(self.text)})' # The __iter__ method is the only addition to the previous Sentence
                                                  # implementation. This version has no __getitem__, to make it clear that the class
                                                  # is iterable because it implements __iter__
  
  def __iter__(self):
    return SentenceIterator(self.words)  # __iter__ fulfills the iterable protocol by instantiating and returning an iterator.
  
  
  class SentenceIterator(self, words):
    
    def __init__(self, words):
      self.words = words
      self.index = index
      
    def __next__(self):
      try:
        word = self.words[self.index]
        except IndexError:
          raise StopIteration()
        self.index += 1
        return word
      
    def __iter__(self):
      return self

### Remark

An iterable should never act as an iterator over itself. In other
words, iterables must implement \_\_iter\_\_, but not \_\_next\_\_.
On the other hand, for convenience, iterators should be iterable.
An iterator’s \_\_iter\_\_ should just return self

## Sentence Take #3: A Generator Function

In [0]:
#Example 14-5. sentence_gen.py: Sentence implemented using a generator function
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
  
  def __init__(self, text):
    self.text = text
    self.words = RE_WORD.findall(text)
    
  def __repr__(self):
    return f'Sentence(reprlib.repr(self.text))'
  
  def __iter__(self):
    for word in self.words:  # Iterate over self.word
      yield word             # yield word 
    return

### Remark
Back in the Sentence code in Example 14-4, \_\_iter\_\_ called the SentenceIterator
constructor to build an iterator and return it. Now the iterator in Example 14-5 is in
fact a generator object, built automatically when the \_\_iter\_\_ method is called, because
\_\_iter\_\_ here is a generator function.

In [0]:
def gen_AB():
  print('start')
  yield 'A'  # yield 'A' in the generator function body produces the value A consumed by
             # the for loop, which gets assigned to the c variable and results in the output --> A.
    
  print('continue')  # Iteration continues with a second call next(g), advancing the generator function
                     # body from yield 'A' to yield 'B'. The text continue is output because of the
                     # second print in the generator function body.
  yield 'B'
  print('end')
  
for c in gen_AB():
  print('-->', c)

start
--> A
continue
--> B
end


## Another Example: Arithmetic Progression Generator

In [0]:
# Example 14-11 lists the implementation of the ArithmeticProgression class
class ArithmeticProgression:
  def __init__(self, begin, step, end=None):
    self.begin = begin
    self.step = step
    self.end = end
    
  def __iter__(self):
    result = type(self.begin + self.step)(self.begin) # This line produces a result value equal to self.begin, but coerced to the type of the subsequent additions.
    forever = self.end is None  # For readability, the forever flag will be True if the self.end attribute is None, resulting in an unbounded series.
    index = 0
    while forever or result < self.end:  # This loop runs forever or until the result matches or exceeds self.end. When this loop exits, so does the function
      yield result  # The current result is produced.
      index += 1
      result = self.begin + self.step * index   # The next potential result is calculated. It may never be yielded, because the while loop may terminate

In [3]:
ap = ArithmeticProgression(0, 1, 3)
list(ap)

[0, 1, 2]

In [4]:
ap = ArithmeticProgression(1, 0.5, 3)
list(ap)

[1.0, 1.5, 2.0, 2.5]

In [5]:
ap = ArithmeticProgression(0, 1/3, 1)
list(ap)

[0.0, 0.3333333333333333, 0.6666666666666666]

In [11]:
# Example 14-12. The aritprog_gen generator function
def aritprog_gen(begin, step, end=None):
  result = type(begin + step)(begin)
  forever = end is None
  while forever or result < end:
    yield result
    index += 1
    result = begin + step * index
    
type(1+ 3)(1)

1

### Arithmetic Progression with itertools
The itertools module in Python 3.4 has 19 generator functions that can be combined
in a variety of interesting ways.
For example, the itertools.count function returns a generator that produces numbers.
Without arguments, it produces a series of integers starting with 0. But you can provide
optional start and step values to achieve a result very similar to our aritprog_gen
functions.

In [13]:
import itertools
gen = itertools.count(1, .2)
type(gen)

itertools.count

In [14]:
next(gen)

1

In [15]:
next(gen)

1.2

In [16]:
next(gen)

1.4

In [0]:
# Example 14-13. aritprog_v3.py: this works like the previous aritprog_gen functions
def aritprog_gen(begin, step, end=None):
  first = type(begin + step)(begin)
  ap_gen = itertools.count(first, step)
  if end is not None:
    ap_gen = itertools.takewhile(lambda n: n < end, ap_gen)
  return ap_gen
    

### Remark
The point of Example 14-13 is: when implementing generators, know what is available
in the standard library, otherwise there’s a good chance you’ll reinvent the wheel.

### Filtering generator functions
1. itertools.compress(it, selector_it): Consumes two iterables in parallel; yields items from it whenever the corresponding item in selector_it is truthy
2. itertools.dropwhile(predicate, it): Consumes it skipping items while predicate computes truthy, then yields every remaining item (no further checks are made)
3. (built-in) filter(predicate,it): Applies predicate to each item of iterable, yielding the item if predicate(item) is truthy; if predicate is None, only truthy items are yielded
4. itertools.filterfalse(predicate, it): Same as filter, with the predicate logic negated: yields items whenever predicate computes falsy
5. itertools.islice(it, stop) or islice(it,start, stop,step=1): Yields items from a slice of it, similar to s[:stop] or s[start:stop:step] except it can be any iterable, and the operation is lazy
6. itertools.takewhile(predicate, it): Yields items while predicate computes truthy, then stops and no further checks are made

In [0]:
# Example 14-14. Filtering generator functions examples
def vowel(c):
  return c.lower() in 'aeiou'

In [26]:
list(filter(vowel, 'Farshid'))


['a', 'i']

In [27]:
import itertools
list(itertools.filterfalse(vowel, 'Farshid'))

['F', 'r', 's', 'h', 'd']

In [33]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [37]:
list(itertools.takewhile(vowel, 'Faaarsaahid'))

[]

In [38]:
list(itertools.compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [39]:
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

In [40]:
list(itertools.islice('Aardvark', 4, 7))

['v', 'a', 'r']

In [41]:
list(itertools.islice('Aardvark', 1, 7, 2))

['a', 'd', 'a']

### Mapping Generator Functions

1. itertools accumulate(it,
[func])
Yields accumulated sums; if func is provided, yields the result of applying it to the
first pair of items, then to the first result and next item, etc.
2. (built-in) enumerate(itera
ble, start=0)
Yields 2-tuples of the form (index, item), where index is counted from
start, and item is taken from the iterable
3. (built-in) map(func, it1,
[it2, …, itN])
Applies func to each item of it, yielding the result; if N iterables are given, func
must take N arguments and the iterables will be consumed in parallel
4. itertools starmap(func, it) Applies func to each item of it, yielding the result; the input iterable should yield
iterable items iit, and func is applied as func(*iit)

In [42]:
### Example 14-15. itertools.accumulate generator function examples
import itertools
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [43]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [44]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [45]:
import operator
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [48]:
list(itertools.accumulate(range(1, 11), operator.add))

[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]

In [49]:
list(itertools.starmap(operator.mul, enumerate('Farshid Balaneji', 1)))

['F',
 'aa',
 'rrr',
 'ssss',
 'hhhhh',
 'iiiiii',
 'ddddddd',
 '        ',
 'BBBBBBBBB',
 'aaaaaaaaaa',
 'lllllllllll',
 'aaaaaaaaaaaa',
 'nnnnnnnnnnnnn',
 'eeeeeeeeeeeeee',
 'jjjjjjjjjjjjjjj',
 'iiiiiiiiiiiiiiii']

In [50]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

list(itertools.starmap(lambda a, b: b/a, enumerate(itertools.accumulate(sample), 1)))

[5.0,
 4.5,
 3.6666666666666665,
 4.75,
 5.2,
 5.333333333333333,
 5.0,
 4.375,
 4.888888888888889,
 4.5]

In [52]:
for a, b in enumerate(itertools.accumulate(sample), 1):
  print(a, b)

1 5
2 9
3 11
4 19
5 26
6 32
7 35
8 35
9 44
10 45


### Generator functions that merge multiple input iterables

1. itertools chain(it1, …, itN) Yield all items from it1, then from it2 etc., seamlessly
2. itertools chain.from_iterable(it) Yield all items from each iterable produced by it, one after the other,
seamlessly; it should yield iterable items, for example, a list of iterables
3. itertools product(it1, …, itN, re
peat=1)
Cartesian product: yields N-tuples made by combining items from each
input iterable like nested for loops could produce; repeat allows the
input iterables to be consumed more than once
4. (built-in) zip(it1, …, itN) Yields N-tuples built from items taken from the iterables in parallel, silently
stopping when the first iterable is exhausted
5. itertools zip_longest(it1, …,
itN, fillvalue=None)
Yields N-tuples built from items taken from the iterables in parallel,
stopping only when the last iterable is exhausted, filling the blanks with
the fillvalue

In [57]:
list(itertools.chain('ABC', range(4), range(3)))

['A', 'B', 'C', 0, 1, 2, 3, 0, 1, 2]

In [58]:
list(itertools.chain(enumerate('ABC')))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [59]:
list(enumerate('ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [60]:
list(itertools.chain.from_iterable(enumerate('ABC')))

[0, 'A', 1, 'B', 2, 'C']

In [66]:
list(zip('ABC', range(4)))

[('A', 0), ('B', 1), ('C', 2)]

In [68]:
list(itertools.zip_longest('ABC', range(5), fillvalue='no_val'))

[('A', 0), ('B', 1), ('C', 2), ('no_val', 3), ('no_val', 4)]

In [69]:
list(itertools.product('ABC', range(2)))

[('A', 0), ('A', 1), ('B', 0), ('B', 1), ('C', 0), ('C', 1)]

In [73]:
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product('JQKA', suits))

[('J', 'spades'),
 ('J', 'hearts'),
 ('J', 'diamonds'),
 ('J', 'clubs'),
 ('Q', 'spades'),
 ('Q', 'hearts'),
 ('Q', 'diamonds'),
 ('Q', 'clubs'),
 ('K', 'spades'),
 ('K', 'hearts'),
 ('K', 'diamonds'),
 ('K', 'clubs'),
 ('A', 'spades'),
 ('A', 'hearts'),
 ('A', 'diamonds'),
 ('A', 'clubs')]

In [74]:
list(itertools.product('ABC', repeat=2))

[('A', 'A'),
 ('A', 'B'),
 ('A', 'C'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'C'),
 ('C', 'A'),
 ('C', 'B'),
 ('C', 'C')]

### Generator functions that expand each input item into multiple output items

1. itertools combinations(it,
out_len)
Yield combinations of out_len items from the items yielded by it
2. itertools combinations_with_re
placement(it, out_len)
Yield combinations of out_len items from the items yielded by it,
including combinations with repeated items
3. itertools count(start=0, step=1) Yields numbers starting at start, incremented by step, indefinitely
4. itertools cycle(it) Yields items from it storing a copy of each, then yields the entire
sequence repeatedly, indefinitely
5. itertools permutations(it,
out_len=None)
Yield permutations of out_len items from the items yielded by it;
by default, out_len is len(list(it))
6. itertools repeat(item, [times]) Yield the given item repeadedly, indefinetly unless a number of times
is given

In [0]:
import itertools
import operator 

ct = itertools.count()

In [3]:
next(ct)

0

In [4]:
next(ct), next(ct), next(ct)

(1, 2, 3)

In [5]:
list(itertools.islice(itertools.count(1, .3), 3))

[1, 1.3, 1.6]

In [0]:
cy = itertools.cycle('ABC')

In [7]:
next(cy)

'A'

In [8]:
list(itertools.islice(cy, 7))

['B', 'C', 'A', 'B', 'C', 'A', 'B']

In [0]:
rp = itertools.repeat(7)


In [10]:
next(rp), next(rp), next(rp)

(7, 7, 7)

In [11]:
list(itertools.repeat(8, 4))

[8, 8, 8, 8]

In [19]:
list(map(operator.mul, range(11),  itertools.repeat(5)))

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

In [24]:
list(itertools.combinations('Farshid', 5))

[('F', 'a', 'r', 's', 'h'),
 ('F', 'a', 'r', 's', 'i'),
 ('F', 'a', 'r', 's', 'd'),
 ('F', 'a', 'r', 'h', 'i'),
 ('F', 'a', 'r', 'h', 'd'),
 ('F', 'a', 'r', 'i', 'd'),
 ('F', 'a', 's', 'h', 'i'),
 ('F', 'a', 's', 'h', 'd'),
 ('F', 'a', 's', 'i', 'd'),
 ('F', 'a', 'h', 'i', 'd'),
 ('F', 'r', 's', 'h', 'i'),
 ('F', 'r', 's', 'h', 'd'),
 ('F', 'r', 's', 'i', 'd'),
 ('F', 'r', 'h', 'i', 'd'),
 ('F', 's', 'h', 'i', 'd'),
 ('a', 'r', 's', 'h', 'i'),
 ('a', 'r', 's', 'h', 'd'),
 ('a', 'r', 's', 'i', 'd'),
 ('a', 'r', 'h', 'i', 'd'),
 ('a', 's', 'h', 'i', 'd'),
 ('r', 's', 'h', 'i', 'd')]

### . Rearranging generator functions

1. itertools groupby(it,
key=None)
Yields 2-tuples of the form (key, group), where key is the grouping criterion
and group is a generator yielding the items in the group

2. (built-in) reversed(seq) Yields items from seq in reverse order, from last to first; seq must be a sequence
or implement the __reversed__ special method
3. itertools tee(it, n=2) Yields a tuple of n generators, each yielding the items of the input iterable
independently

In [26]:
string = 'AAFGAHHHGFDCVFDDC'
list(itertools.groupby(string))

[('A', <itertools._grouper at 0x7f255e6884e0>),
 ('F', <itertools._grouper at 0x7f255e688dd8>),
 ('G', <itertools._grouper at 0x7f255e688630>),
 ('A', <itertools._grouper at 0x7f255e688c18>),
 ('H', <itertools._grouper at 0x7f255e688f98>),
 ('G', <itertools._grouper at 0x7f255e688a58>),
 ('F', <itertools._grouper at 0x7f255e688b38>),
 ('D', <itertools._grouper at 0x7f255e6881d0>),
 ('C', <itertools._grouper at 0x7f255e688390>),
 ('V', <itertools._grouper at 0x7f255e688908>),
 ('F', <itertools._grouper at 0x7f255e6888d0>),
 ('D', <itertools._grouper at 0x7f255e688f60>),
 ('C', <itertools._grouper at 0x7f255e6886d8>)]

In [27]:
for char, group in itertools.groupby(string):
  print(char, '->', list(group))

A -> ['A', 'A']
F -> ['F']
G -> ['G']
A -> ['A']
H -> ['H', 'H', 'H']
G -> ['G']
F -> ['F']
D -> ['D']
C -> ['C']
V -> ['V']
F -> ['F']
D -> ['D', 'D']
C -> ['C']


In [28]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 'bat', 'dolphin', 'shark', 'lion']
animals.sort(key=len) # To use groupby, the input should be sorted; here the words are sorted by length.
animals

['rat', 'bat', 'duck', 'bear', 'lion', 'eagle', 'shark', 'giraffe', 'dolphin']

In [29]:
for length, group in itertools.groupby(animals, len):
  print(length, '-->', list(group))

3 --> ['rat', 'bat']
4 --> ['duck', 'bear', 'lion']
5 --> ['eagle', 'shark']
7 --> ['giraffe', 'dolphin']


In [30]:
for length, group in itertools.groupby(reversed(animals), len): 
  print(length, '-->', list(group))

7 --> ['dolphin', 'giraffe']
5 --> ['shark', 'eagle']
4 --> ['lion', 'bear', 'duck']
3 --> ['bat', 'rat']


### New Syntax in Python 3.3: yield from

Nested for loops are the traditional solution when a generator function needs to yield
values produced from another generator.

In [2]:
def chain(*iterables):
  for it in iterables:
    for i in it:
      yield i
      
s = 'AbcFd'
l = [1, 2, 3]
t = tuple(range(5))
list(chain(s, l, t))

['A', 'b', 'c', 'F', 'd', 1, 2, 3, 0, 1, 2, 3, 4]

The chain generator function is delegating to each received iterable in turn. PEP 380
— Syntax for Delegating to a Subgenerator introduced new syntax for doing that

In [3]:
def chain(*iterables):
  for i in iterables:
    yield from i
    
list(chain(s, l, t))

['A', 'b', 'c', 'F', 'd', 1, 2, 3, 0, 1, 2, 3, 4]

### Remark

As you can see, __yield from__ i replaces the inner for loop completely. The use of yield
from in this example is correct, and the code reads better, but it seems like mere syntactic
sugar. Besides replacing a loop, yield from creates a channel connecting the inner
generator directly to the client of the outer generator. This channel becomes really im‐
portant when generators are used as coroutines and not only produce but also consume
values from the client code

## Iterable Reducing Functions
#### Built-in functions that read iterables and return single values
1. (built-in) all(it) Returns True if all items in it are truthy, otherwise False; all([])
returns True
2. (built-in) any(it) Returns True if any item in it is truthy, otherwise False; any([])
returns False
3. (built-in) max(it, [key=,] [de
fault=])
Returns the maximum value of the items in it;a key is an ordering
function, as in sorted; default is returned if the iterable is empty
4. (built-in) min(it, [key=,] [de
fault=])
Returns the minimum value of the items init.bkeyis an ordering function,
as in sorted; default is returned if the iterable is empty
5. functools reduce(func, it, [ini
tial])
Returns the result of applying func to the first pair of items, then to that
result and the third item and so on; if given, initial forms the initial
pair with the first item
6. (built-in) sum(it, start=0) The sum of all items in it, with the optional start value added (use
math.fsum for better precision when adding floats)